In [ ]:
!/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install opencv-python
!/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install pyzbar
%pip install pyzbar
%pip install --upgrade google-cloud-bigquery
%brew install zbar

In [ ]:
import sys
print(sys.executable)
import cv2
import pyzbar.pyzbar
from google.cloud import bigquery
from datetime import datetime
import threading
import time


In [21]:
import cv2
from google.cloud import bigquery
from google.cloud.bigquery import SchemaField
from datetime import datetime
import threading
import time
import os
import json
#will probably make new table soon when finished testing
project_id = 'finch-project-399922'
dataset_id = 'finch_beta_table'  
table_id_file = 'table_id.txt'
#will either make new table w datetime name or use old one if id file exists
if os.path.exists(table_id_file):
    with open(table_id_file, 'r') as file:
        table_id = file.read().strip()
else:
    table_id = 'test1_' + datetime.now().strftime('%Y%m%d_%H%M%S')
    with open(table_id_file, 'w') as file:
        file.write(table_id)

full_table_id = f"{project_id}.{dataset_id}.{table_id}"

#client stuff can be silly w gcloud. If you ever get errors w location or project id or credentials,
#it's often easiest to specify with the api client you're using
client = bigquery.Client(project=project_id)
batched_data = []
data_lock = threading.Lock()
upload_interval = 10  

schema = [
    SchemaField("qr_data", "STRING", mode="REQUIRED"),
    SchemaField("timestamp", "STRING", mode="REQUIRED"),
    SchemaField("Position", "STRING", mode="NULLABLE"),
]


table = bigquery.Table(full_table_id, schema=schema)
table = client.create_table(table, exists_ok=True)  

def insert_rows_to_bigquery():
    while True:
        time.sleep(upload_interval)
        with data_lock:
            if not batched_data:
                continue
            data_to_insert = batched_data.copy()
            batched_data.clear()

        rows_to_insert = [{"qr_data": data, "timestamp": timestamp.isoformat(), "Position": position} for data, timestamp, position in data_to_insert]

        errors = client.insert_rows_json(table, rows_to_insert)  
        if errors:
            print(f"Encountered errors while inserting rows: {errors}")
        else:
            print(f"Data uploaded to {full_table_id} at {datetime.utcnow().isoformat()}")

def main():
    #upload function is constantly running, upload every 10 sec currently
    upload_thread = threading.Thread(target=insert_rows_to_bigquery, daemon=True)
    upload_thread.start()

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Could not open webcam")
        exit()

    qr_code_detector = cv2.QRCodeDetector()

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        val, pts, qr_code = qr_code_detector.detectAndDecode(frame)
        
        if val:
            print("QR Data: ", val)
            #needed to convert positional data or else it's wonky
            position_str = json.dumps(pts.tolist()) if pts is not None else None
            with data_lock:
                batched_data.append((val, datetime.utcnow(), position_str))

        cv2.imshow("QR Code Scanner", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()


QR Data:  50
QR Data:  50
QR Data:  50
QR Data:  50
QR Data:  50
QR Data:  50
QR Data:  50
QR Data:  50
QR Data:  50
QR Data:  50
QR Data:  50
QR Data:  50
QR Data:  50
QR Data:  50
QR Data:  50
QR Data:  50
Data uploaded to finch-project-399922.finch_beta_table.test1_20230923_192158 at 2023-09-24T12:11:37.425049


KeyboardInterrupt: 